In [1]:
!pip install -U transformers==4.44.2 datasets tensorboard sentencepiece accelerate evaluate rouge_score --quiet bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 118.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the pack

In [2]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install unsloth
# Get latest Unsloth
!pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, AutoModelForCausalLM
from functools import partial
from datasets import Dataset
import pandas as pd
import numpy as np
import evaluate
import warnings
import torch

from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
warnings.simplefilter("ignore", UserWarning)

In [6]:
from google.colab import drive
drive.mount('/content/drive')
datasetPath="/content/drive/My Drive/Dataset/CLEANDATA.csv"

data = pd.read_csv(datasetPath,encoding = 'unicode_escape')
data.drop_duplicates(keep='first')
df = data[['flatten_article','flatten_summary']]

df = df.rename(columns={'flatten_article':'document', 'flatten_summary':'summary'})


Mounted at /content/drive


In [7]:
df = df.sample(frac=1).reset_index(drop=True)
train = df[:800]
valid = df[800:]
test = df[800:]
train.shape, valid.shape,test.shape

df_train=Dataset.from_pandas(train)
df_valid=Dataset.from_pandas(valid)

instruct= """
Ringkaslah dokumen sesuai dengan aturan berikut:
1. ringkas menggunakan teknik ringkasan ekstraksi, yaitu Ringkasan harus terdiri dari kalimat-kalimat dari dokumen asli.
2. ringkasan maksimal 85 kata.
contoh dari proses ringkasan:
Dokumen : Pebalap Yamaha Valentino Rossi punya pandangan tersendiri terhadap sukses rivalnya , Marc Marquez dalam mengarungi musim balap MotoGP 2017 . Marquez kini berada di puncak klasemen dengan 129 poin meskipun hanya dua kali menang di seri balapan . Rossi menganggap strategi yang dibuat Marquez dan timnya di Honda cukup pintar . Pasalnya , jika dianalisis dari sembilan seri yang sudah berlangsung , Marquez hanya dua kali menang , dua kali gagal finis , dua kali menempati podium kedua , sekali podium ketiga , keempat dan keenam . Berbeda dengan rekan Marquez , rekan setim Rossi , Maverick Vinales sudah mengantongi tiga kali kemenangan . Namun , Marquez dinilai lebih unggul dalam memaksimalkan poin dibandingkan Vinales dan Rossi , kala tidak memenangkan sejumlah seri pada paruh pertama musim ini . Selain hasil tiga kemenangan , Vinales hanya meraih podium di peringkat kedua , posisi keempat , keenam , ke - 10 , dan dua kali gagal finis . Raihan itu tak sebagus Marquez saat gagal meraih kemenangan . Selain itu , faktor lain yang membuat Marquez bisa punya hasil lebih baik yakni lantaran kuda besi Honda RC213V yang ditungganginya disebut jarang mengalami kerusakan . Marquez kini unggul lima poin dari Vinales yang menguntit di belakangnya serta selisih 10 poin dari Rossi usai menang di Sirkuit Sachsenring , Jerman awal Juli lalu . Buat Rossi , sukses yang diraih Marquez itu adalah buah dari ketekunan dan usaha yang dilakukan sang ' Baby Alien ' selama ini . Bahkan , ketika kondisi motornya tidak baik , ia bisa tetap mempertahankan posisinya di klasemen kejuaraan dunia . " Marquez memimpin karena pada saat dia berada dalam masalah , dia hanya kehilangan lebih sedikit poin daripada yang kami alami saat sedang dalam kondisi sulit , " kata Rossi dikutip Autosport , Kamis ( 13 / 7 ) . " Ketika dia harus menang dan yakin , ia bisa menang . Tapi saat dia berjuang dengan keadaan sulit , dia mampu membatasi penampilan atas kerusakan mesinnya dengan sangat baik dan memilih cari aman . Karena itulah dia memimpin , " tukasnya .
hasil ringkasan: Pebalap Yamaha Valentino Rossi punya pandangan tersendiri terhadap sukses rivalnya , Marc Marquez dalam mengarungi musim balap MotoGP 2017 . Marquez kini berada di puncak klasemen dengan 129 poin . Rossi menganggap strategi yang dibuat Marquez dan timnya di Honda cukup pintar . Pasalnya , dari sembilan seri yang sudah berlangsung , Marquez hanya dua kali menang , dua kali gagal finis .
"""

df_train = df_train.add_column("task", [instruct] * len(df_train))
df_valid = df_valid.add_column("task", [instruct] * len(df_valid))


In [8]:
print(df_train[0])

{'document': ' Menghadapi lomba renang Wakil Gubernur DKI Jakarta Sandiaga Uno dengan Menteri Kelautan dan Perikanan , Susi Pudjiastuti yang akan dilangsungkan pada 25 Februari 2018 mendatang berbagai persiapan dilakukan . Salah satunya dengan mengerahkan puluhan pasukan Kuning dari Satgas Sudin Bina Marga , Jakarta Utara untuk memperbaiki trotoar yang ada disepanjang Jalan Danau Sunter Selatan , Sunter Agung , Tanjung Priok , Jakarta Utara . Kasie Jalan dan Jembatan , Sudin Bina Marga , Jakarta Utara , Ujang Royani mengakui pihaknya saat ini terus ngebut memperbaiki fasilitas yang ada di sepanjang Jalan Dansu Sunter Selatan , Sunter Agung , Tanjung Priok . Perbaikan ini dilakukan karena selain kondisi sudah mulai rusak juga untuk memberi rasa aman dan nyaman bagi warga . Saat ini puluhan tim Kuning dari Sudin Bina Marga kami kerahkan ke lokasi , untuk mempercepat pembaikan . Kami berharap menjelang pelaksanaan lomba renang nanti akan selesai seluruhnya , terang Ujang Royani , Sabtu ( 

In [9]:
max_seq_length = 541 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.3.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.7 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [11]:
from unsloth import to_sharegpt
dataset_train = to_sharegpt(
    df_train,
    merged_prompt =
        "{task}[[Berikut dokumen: {document}.]]",
    conversation_extension = 5, # Randomnly combines conversations into 1! Good for long convos
    output_column_name = "summary",
)


dataset_valid = to_sharegpt(
    df_valid,
    merged_prompt =
        "{task}[[Berikut dokumen: {document}.]]",
    conversation_extension = 5, # Randomnly combines conversations into 1! Good for long convos
    output_column_name = "summary",
)

Merging columns:   0%|          | 0/800 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/800 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/800 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/800 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/800 [00:00<?, ? examples/s]

...

Flattening the indices:   0%|          | 0/200 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/200 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/200 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/200 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/200 [00:00<?, ? examples/s]

In [12]:
from unsloth import standardize_sharegpt
dataset_train = standardize_sharegpt(dataset_train)
dataset_valid = standardize_sharegpt(dataset_valid)

Standardizing format:   0%|          | 0/800 [00:00<?, ? examples/s]

Standardizing format:   0%|          | 0/200 [00:00<?, ? examples/s]

In [13]:
from pprint import pprint
pprint(dataset_train)

Dataset({
    features: ['conversations'],
    num_rows: 800
})


In [14]:
chat_template = """
anda merupakan asisten pribadi untuk melakukan ringkasan extraksi pada document yang diberikan
>>> Document:
{INPUT}
>>> Hasil ringkasan:
{OUTPUT}
"""

from unsloth import apply_chat_template
dataset_train = apply_chat_template(
    dataset_train,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)
dataset_valid = apply_chat_template(
    dataset_valid,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [15]:
pprint(dataset_train[1])


{'conversations': [{'content': '\n'
                               'Ringkaslah dokumen sesuai dengan aturan '
                               'berikut:\n'
                               '1. ringkas menggunakan teknik ringkasan '
                               'ekstraksi, yaitu Ringkasan harus terdiri dari '
                               'kalimat-kalimat dari dokumen asli.\n'
                               '2. ringkasan maksimal 85 kata.\n'
                               'contoh dari proses ringkasan:\n'
                               'Dokumen : Pebalap Yamaha Valentino Rossi punya '
                               'pandangan tersendiri terhadap sukses rivalnya '
                               ', Marc Marquez dalam mengarungi musim balap '
                               'MotoGP 2017 . Marquez kini berada di puncak '
                               'klasemen dengan 129 poin meskipun hanya dua '
                               'kali menang di seri balapan . Rossi menganggap '
               

In [16]:
dataset_train

Dataset({
    features: ['conversations', 'text'],
    num_rows: 800
})

In [17]:
pprint(dataset_valid)


Dataset({
    features: ['conversations', 'text'],
    num_rows: 200
})


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("device yang digunakan (GPU-cuda/CPU):", device)

device yang digunakan (GPU-cuda/CPU): cuda


In [19]:
def decode_predictions(eval_pred, tokenizer, device):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids

    predictions = torch.tensor(predictions).to(device)
    labels = torch.tensor(labels).to(device)

    decoded_preds = tokenizer.batch_decode(predictions.tolist(), skip_special_tokens=True)
    labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels.tolist(), skip_special_tokens=True)

    return decoded_preds, decoded_labels, predictions, labels

In [20]:
rouge = evaluate.load("rouge")

In [21]:
def calculate_rouge(decoded_preds, decoded_labels, predictions, tokenizer, device, rouge):
    pad_token_id = torch.tensor(tokenizer.pad_token_id, device=device)

    with torch.no_grad():
        result = rouge.compute(
            predictions=decoded_preds,
            references=decoded_labels,
            use_stemmer=True,
            rouge_types=['rouge1', 'rouge2', 'rougeL']
        )

    prediction_lens = [torch.count_nonzero(pred != pad_token_id).item() for pred in predictions]
    result["gen_len"] = torch.tensor(prediction_lens, device=device).float().mean().item()

    return {k: round(v, 4) for k, v in result.items()}

In [22]:
def compute_metrics(eval_pred):
    decoded_preds, decoded_labels, predictions, labels = decode_predictions(eval_pred, tokenizer, device)
    result = calculate_rouge(decoded_preds, decoded_labels, predictions, tokenizer, device, rouge)
    return result

In [23]:
def preprocess_logits_for_metrics(logits, labels):
    pred_ids = torch.argmax(logits, dim=-1)
    return pred_ids, labels

In [24]:
def rouge_score_rows(eval_pred, tokenizer, device, rouge):
    decoded_preds, decoded_labels, predictions, labels = decode_predictions(eval_pred, tokenizer, device)

    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    for pred, label in zip(decoded_preds, decoded_labels):
        scores = rouge.compute(predictions=[pred], references=[label])
        rouge1_scores.append(round(scores['rouge1'], 4))
        rouge2_scores.append(round(scores['rouge2'], 4))
        rougeL_scores.append(round(scores['rougeL'], 4))

    rouge_score_row = pd.DataFrame({
        'Prediksi Summary': decoded_preds,
        'Aktual Summary': decoded_labels,
        'rouge1': rouge1_scores,
        'rouge2': rouge2_scores,
        'rougeL': rougeL_scores
    })

    return rouge_score_row

In [25]:
!pip install wandb

In [26]:
import wandb

# Log in to W&B - you'll be prompted to input your API key
wandb.login()

# Set W&B environment variables
%env WANDB_WATCH=all
%env WANDB_SILENT=true

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_WATCH=all
env: WANDB_SILENT=true


In [27]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from transformers.utils import logging
from transformers import EarlyStoppingCallback


project_name = "llama3.2-indosum"
entity = "wandb"
wandb.init(project=project_name, name="unsloth-llama3.2")


gradient_accumulation=4
per_device_batch_size=2
learning_rate=1e-5#2e-4
OUT_DIR = 'llama3.2_finetune_exsum2'

WARMUP_STEPS=200
MAX_STEPS=500
epoch=5

training_args = TrainingArguments(
        per_device_train_batch_size = per_device_batch_size,
        gradient_accumulation_steps = gradient_accumulation,
        num_train_epochs=epoch,
        warmup_steps = WARMUP_STEPS,
        learning_rate = learning_rate,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        eval_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        logging_steps=0,
        optim = "adamw_8bit",
        report_to="wandb",
        weight_decay = 0.03,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = OUT_DIR,
        load_best_model_at_end=True,  # Tambahkan ini
        metric_for_best_model="eval_loss",  # Tambahkan ini, atau gunakan metrik lain
        greater_is_better=False,  # Tambahkan ini karena loss lebih baik jika nilainya lebih kecil

)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=training_args,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=3)  # Early stopping based on validation loss
    ],
)

logging.set_verbosity_info()

# Initialize W&B


wandb: Currently logged in as: adjiekhalis (adjiekhalis-universitas-islam-riau-). Use `wandb login --relogin` to force relogin


Map (num_proc=2):   0%|          | 0/800 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

In [28]:


history = trainer.train()
wandb.finish()



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 800 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 500
 "-____-"     Number of trainable parameters = 11,272,192
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
1,2.663900,2.429217,0.302300,0.062300,0.244200,541.000000
2,1.483200,0.279866,0.936100,0.894300,0.936100,541.000000
3,0.064000,0.030792,0.994400,0.994400,0.994400,541.000000
4,0.029400,0.028640,0.994400,0.994400,0.994400,541.000000
5,0.028400,0.028381,0.994400,0.994400,0.994400,541.000000



***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to llama3.2_finetune_exsum2/checkpoint-100
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--Llama-3.2-1B-bnb-4bit/snapshots/87140e4eefb42036dd1b967d54f6cbffa828c932/config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.2-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit

eval/gen_len,▁▁▁▁▁
eval/loss,█▂▁▁▁
eval/rouge1,▁▇███
eval/rouge2,▁▇███
eval/rougeL,▁▇███
eval/runtime,▁▂▂█▂
eval/samples_per_second,█▇▇▁▇
eval/steps_per_second,█▇▇▁▇
train/epoch,▁▁▃▃▅▅▆▆███
train/global_step,▁▁▃▃▅▅▆▆███
train/grad_norm,█▇▁▁▁


In [29]:
model.save_pretrained(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--Llama-3.2-1B-bnb-4bit/snapshots/87140e4eefb42036dd1b967d54f6cbffa828c932/config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.2-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload"

('llama3.2_finetune_exsum2/tokenizer_config.json',
 'llama3.2_finetune_exsum2/special_tokens_map.json',
 'llama3.2_finetune_exsum2/tokenizer.json')

In [30]:
model_name="/content/drive/MyDrive/Dataset/Llama-3.2-3B-bnb-4bit"

model.save_pretrained(model_name)
tokenizer.save_pretrained(model_name)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--Llama-3.2-1B-bnb-4bit/snapshots/87140e4eefb42036dd1b967d54f6cbffa828c932/config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.2-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload"

('/content/drive/MyDrive/Dataset/Llama-3.2-3B-bnb-4bit/tokenizer_config.json',
 '/content/drive/MyDrive/Dataset/Llama-3.2-3B-bnb-4bit/special_tokens_map.json',
 '/content/drive/MyDrive/Dataset/Llama-3.2-3B-bnb-4bit/tokenizer.json')

In [ ]:





























































































































eval_predictions = trainer.predict(dataset_valid)
rouge_score_row = rouge_score_rows(eval_predictions, tokenizer, device, rouge)

In [ ]:
rouge_score_row

In [ ]:
print(valid.iloc[0]['document'])

In [ ]:
prediksi_summary = rouge_score_row.loc[0, 'Prediksi Summary']

In [ ]:
print(f"Prediksi Summary : {prediksi_summary}")

In [ ]:
aktual_summary = rouge_score_row.loc[0, 'Aktual Summary']

In [ ]:
print(f"Aktual Summary : {aktual_summary}")

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'], use_stemmer=True)
scores = scorer.score(prediksi_summary, aktual_summary)

for score_type, score in scores.items():
    print(f"{score_type}: precision: {score.precision:.4f}, recall: {score.recall:.4f}, fmeasure: {score.fmeasure:.4f}")

In [ ]:
rouge_score_row.to_csv('cek_hasil_indosum_data_2.csv', index=False)